# Optical Flow

In this notebook you'll use *Optical Flow* to track features produced by *Shi-Tomasi*, predicting where the features will be in the next frame. This difference in pixel location is the velocity measured in pixels/frame.

In [ ]:
%matplotlib inline

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from moviepy.editor import ImageSequenceClip
import imageio

In [ ]:
plt.rcParams['figure.figsize'] = 12, 12

Similar to *Shi-Tomasi*, the *Optical Flow* algorithm has many tunable paramters. We'll be using [cv.calcOpticalFlowPyrLK](https://docs.opencv.org/3.4.1/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323) which uses the *Lucas-Kanade* method. Once again, there are several parameters to tune.

In [ ]:
# Parameters for Shi Tamasi features
feature_params = dict(maxCorners=0,  # no limit on number of corners
                      qualityLevel=0.05,
                      minDistance=50,
                      blockSize=7)

# Parameters for Lucas Kanade optical flow
optical_flow_params = dict(winSize=(?, ?),
                           maxLevel=?, 
                           criteria=(?, ?, ?))

You may want to shorten the loop in `track` while you're fiddling with the parameters since it'll shorten the time to create the video, and hence the development time. Also note the `detect_interval` argument, by default corners are redetected on every 5th frame.

In [ ]:
def shi_tomasi(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return cv.goodFeaturesToTrack(gray, **feature_params)


def optical_flow(frame0, frame1, corners0):        
    # convert images to grayscale
    frame0_gray = cv.cvtColor(frame0, cv.COLOR_BGR2GRAY)
    frame1_gray = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

    # TODO: Use the Lucas-Kanade method `cv.calcOpticalFlowPyrLK` for Optical Flow
    # Indices of the `status` array which equal 1 signify a corresponding new feature has been found
    corners1, status, err = ?
    
    return corners1, status==1
    

def track(reader, detect_interval=5):
    frames = []
    
    frame0 = reader.get_data(0)
    
    # Initial corners, after this we'll detect
    # corners on the interval `detect_interval`
    corners0 = shi_tomasi(frame0)
    
    mean_u = 0
    mean_v = 0
    
    # Used for weighted average update of the velocity
    alpha = 0.97
        
    # NOTE: You may want to limit this for loop
    # to a shorter range at first.
    for i in range(1, reader.get_length()):
        frame1 = reader.get_data(i)
        # for visualization
        vis = frame1.copy()         
        
        corners1, valid = optical_flow(frame0, frame1, corners0)
        
        # This discards any pixels from `corners0` which did not
        # produce a corresponding pixel with optical flow
        velocity = ((corners1 - corners0)[valid==1]).reshape(-1, 2)
        
        # TODO: calculate mean velocity in pixels/frame
        u, v = ?
        
        # NOTE: we use a simple weighted average method
        # but you may want to use some of the
        # estimation techniques you've learned.
        mean_u = alpha * mean_u + (1-alpha) * np.mean(u)
        mean_v = alpha * mean_v + (1-alpha) * np.mean(v)
        
        # Velocity related visuals
        cv.putText(vis, "Mean X Velocity (U) = {0:.2f}".format(mean_u), 
                   (20, 20), cv.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), thickness=2, lineType=cv.LINE_AA)
        cv.putText(vis, "Mean Y Velocity (V) = {0:.2f}".format(mean_v), 
                   (20, 35), cv.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), thickness=2, lineType=cv.LINE_AA)
        vis = cv.arrowedLine(vis, (50, 100), (int(50+5*mean_u), int(100-5*mean_v)), 
                             (0, 255, 0), 2, tipLength=0.3, line_type=cv.LINE_AA)

        # carry over new corners
        corners0 = corners1
        
        # refresh corners
        # If we only relied on corners carrying over
        # we would eventually run out of corners
        if i % detect_interval == 0:
            corners0 = shi_tomasi(frame0)
    
        frame0 = frame1
        frames.append(vis)
        
    return frames

In [ ]:
reader = imageio.get_reader('vid.mp4')

In [ ]:
%time frames = track(reader, detect_interval=5)
print(len(frames), frames[0].shape)

In [ ]:
clip = ImageSequenceClip(frames, fps=24)
clip.ipython_display()

[Solution](./Optical-Flow-Solution.ipynb)